In [217]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import *
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor, Pool
from ngboost import NGBRegressor

In [218]:
train = pd.read_csv("train.csv")
test = pd.read_csv('test.csv')
age = pd.read_csv('age_gender_info.csv')

In [219]:
out_idx = ['C1036',
 'C1095',
 'C1218',
 'C1397',
 'C1502',
 'C1649',
 'C1894',
 'C1988',
 'C2051',
 'C2085',
 'C2431',
 'C2483']

In [220]:
train = train.query('단지코드 not in @out_idx')

In [221]:
reg_list1 = list(train.지역.unique())
reg_list2 = [i for i in reg_list1 if i[-1] == '시']

In [222]:
reg_list2

['대전광역시', '광주광역시', '부산광역시', '울산광역시', '대구광역시', '서울특별시', '세종특별자치시']

In [223]:
age_list1 = list(age.columns)
age_list1.remove('지역')
age_list2 = ['10대미만'] + [str(i) + '대' for i in range(10, 110, 10)]
age_list3 = ['남자', '여자']

In [224]:


sup_list1 = list(train.공급유형.unique())
sup_list2 = [i + '세대수' for i in sup_list1]
i = 1
are_list1 = ['면적' + str(i) + '_' + str(j)  for j in range(10 // i, 600 // i + 1, 1)]
i = 5
are_list2 = ['면적' + str(i) + '_' + str(j)  for j in range(10 // i, 600 // i + 1, 1)]
i = 10
are_list3 = ['면적' + str(i) + '_' + str(j)  for j in range(10 // i, 600 // i + 1, 1)]

In [225]:
data = train.copy()
feature = data.drop_duplicates(['단지코드', '총세대수'])[['단지코드', '지역', '총세대수', '공가수', '단지내주차면수', '등록차량수']]

In [226]:
feature = pd.merge(feature, data.groupby('단지코드')['전용면적별세대수'].sum().reset_index(name = '총세대수2'), how = 'left', on = '단지코드')

In [227]:
for i in reg_list1:
    feature[i] = (feature['지역'] == i) * 1
feature['도시'] = 0

#### '시'로 끝나는 지역은 도시로 간주

In [228]:
for i in reg_list2:
    feature['도시'] += feature[i]

In [229]:
#나이 비율
feature = pd.merge(feature, age, how = 'left', on = '지역')
for i in age_list2:
    feature[i] = 0
    for j in age_list3:
        feature[i] += feature[i + '(' + j + ')']

In [230]:
for i in age_list3:
    feature[i] = 0
    for j in age_list2:
        feature[i] += feature[j + '(' + i + ')']

In [231]:
for i in sup_list1:
    tmp = data[data['공급유형'] == i].groupby(['단지코드', '공급유형'])['전용면적별세대수'].sum().reset_index(name = i + '세대수')
    tmp[i] = 1
    tmp = tmp.drop('공급유형', axis = 1)
    feature = pd.merge(feature, tmp, how = 'left', on = '단지코드').fillna(0)

In [232]:
#공가수
feature['공가비율'] = feature['공가수'] / feature['총세대수2']

In [233]:
tmp = data.copy()
tmp['총전용면적'] = tmp['전용면적'] * tmp['전용면적별세대수']
tmp = tmp.groupby(['단지코드'])['총전용면적'].sum().reset_index(name = '총전용면적')
feature = pd.merge(feature, tmp, how = 'left', on = '단지코드').fillna(0)
feature['평균전용면적'] = feature['총전용면적'] / feature['총세대수2']

In [234]:
tmp = data.copy()
tmp = tmp[tmp['임대건물구분'] == '상가']
tmp['상가총전용면적'] = tmp['전용면적'] * tmp['전용면적별세대수']
tmp = tmp.groupby(['단지코드'])['상가총전용면적'].sum().reset_index(name = '상가총전용면적')
feature = pd.merge(feature, tmp, how = 'left', on = '단지코드').fillna(0)
feature['상가평균전용면적'] = feature['상가총전용면적'] / feature['총세대수2']

In [235]:
tmp = data.copy()
tmp = tmp[tmp['임대건물구분'] == '아파트']
tmp['아파트총전용면적'] = tmp['전용면적'] * tmp['전용면적별세대수']
tmp = tmp.groupby(['단지코드'])['아파트총전용면적'].sum().reset_index(name = '아파트총전용면적')
feature = pd.merge(feature, tmp, how = 'left', on = '단지코드').fillna(0)
feature['아파트평균전용면적'] = feature['아파트총전용면적'] / feature['총세대수2']

In [236]:
for i in [1, 5, 10]:
    tmp = data.copy()
    tmp['전용면적'] = np.round(tmp['전용면적'] / i)
    tmp = tmp.groupby(['단지코드', '전용면적'])['전용면적별세대수'].sum().reset_index(name = '세대수')
    for j in range(10 // i, 600 // i + 1, 1):
        tmp2 = tmp[tmp['전용면적'] == j]
        tmp2['면적' + str(i) + '_' + str(j)] = tmp2['세대수']
        tmp2 = tmp2[['단지코드', '면적' + str(i) + '_' + str(j)]]
        feature = pd.merge(feature, tmp2, how = 'left', on = '단지코드').fillna(0)

In [237]:
#feature['등록차량수'] = (feature['등록차량수'] - feature['단지내주차면수'])
feature = feature.drop('지역', axis = 1)
feature = feature.set_index('단지코드')

In [238]:
i = 1
are_list1 = ['면적' + str(i) + '_' + str(j)  for j in range(84 // i, 85 // i + 1, 1)]

In [239]:
new_y = np.array(feature['등록차량수'] / feature['단지내주차면수'])
parking_able = np.array(feature['단지내주차면수'])
car_cnt = np.array(feature['등록차량수'])

In [240]:
feature_names_ = ['평균전용면적', '공가수', '40대', '50대']
train_feature = np.array(feature[feature_names_])

In [241]:
random_state_ = 42
n_estimators_ = 1000
learning_rate_ = 0.002

In [242]:
kf = KFold(n_splits = 8, shuffle = True, random_state = 42)

In [243]:
for i in range(5):
    scores = []
    for tr_idx, val_idx in kf.split(train_feature):
        tr_X, val_X = train_feature[tr_idx], train_feature[val_idx]
        tr_y, val_y = new_y[tr_idx], new_y[val_idx]
        v1, v2 = parking_able[val_idx], car_cnt[val_idx]
        rgr = LGBMRegressor(n_estimators = 750,
                        random_state = 42,
                        learning_rate=learning_rate_)

        rgr.fit(tr_X, tr_y)
        score = mean_absolute_error(val_y, rgr.predict(val_X) * v1)
        scores.append(score)
    print("LGBM MAE: %0.4f (+/- %0.4f)" %(np.mean(scores), np.std(scores) * 2))
    learning_rate_ += 0.001

LGBM MAE: 570.3778 (+/- 107.4974)
LGBM MAE: 570.2922 (+/- 112.2254)
LGBM MAE: 569.7740 (+/- 114.8336)
LGBM MAE: 569.0669 (+/- 115.9308)
LGBM MAE: 568.7228 (+/- 117.4491)


In [244]:
feature_names_ = ['남자', '10대미만', '20대', '30대', '40대', '국민임대', '공가수', 
                  '상가평균전용면적', '아파트평균전용면적', '도시', 
                  '면적1_26', '면적1_31', '면적1_32', '면적1_34', '면적1_36', '면적1_39', '면적1_40', '면적1_47', '면적1_52', '면적1_85', 
                  '10대미만(여자)', '10대미만(남자)', '10대(여자)', '10대(남자)', '20대(여자)', '30대(남자)', '40대(남자)', 
                  '50대(여자)', '50대(남자)', '60대(여자)', '60대(남자)', 
                  '70대(여자)', '70대(남자)']

In [245]:
train_feature = np.array(feature[feature_names_])

In [246]:
for i in range(5):
    scores = []
    for tr_idx, val_idx in kf.split(train_feature):
        tr_X, val_X = train_feature[tr_idx], train_feature[val_idx]
        tr_y, val_y = train_target[tr_idx], train_target[val_idx]
        v1, v2 = parking_able[val_idx], car_cnt[val_idx]
        rgr = LGBMRegressor(n_estimators = 750,
                        random_state = 42,
                        learning_rate=learning_rate_)

        rgr.fit(tr_X, tr_y)
        score = mean_absolute_error(val_y, rgr.predict(val_X) * v1)
        scores.append(score)
    print("LGBM MAE: %0.4f (+/- %0.4f)" %(np.mean(scores), np.std(scores) * 2))
    learning_rate_ += 0.001

LGBM MAE: 101.5947 (+/- 38.2226)
LGBM MAE: 103.2620 (+/- 39.0320)
LGBM MAE: 103.9929 (+/- 39.3878)
LGBM MAE: 103.6314 (+/- 40.0605)
LGBM MAE: 105.4851 (+/- 40.8031)


In [247]:
md = 4
for i in range(5):
    scores = []
    for tr_idx, val_idx in kf.split(train_feature):
        tr_X, val_X = train_feature[tr_idx], train_feature[val_idx]
        tr_y, val_y = train_target[tr_idx], train_target[val_idx]
        v1, v2 = ddd[val_idx], eee[val_idx]
        rf = RandomForestRegressor(random_state = 729, criterion = 'mae', max_depth = md)

        rf.fit(tr_X, tr_y)
        score = mean_absolute_error(val_y, rf.predict(val_X) * v1)
        scores.append(score)
    print("RF MAE: %0.4f (+/- %0.4f)" %(np.mean(scores), np.std(scores) * 2))
    md += 1

RF MAE: 79.9624 (+/- 31.0014)
RF MAE: 82.6789 (+/- 32.0785)
RF MAE: 85.4007 (+/- 34.8097)
RF MAE: 86.7126 (+/- 33.6916)
RF MAE: 88.7598 (+/- 36.6624)


In [248]:
md = 4
for i in range(5):
    scores = []
    for tr_idx, val_idx in kf.split(train_feature):
        tr_X, val_X = train_feature[tr_idx], train_feature[val_idx]
        tr_y, val_y = train_target[tr_idx], train_target[val_idx]
        v1, v2 = ddd[val_idx], eee[val_idx]
        cb = CatBoostRegressor(iterations = 1000, max_depth = md, random_state = 42, learning_rate = .001, verbose = 0)

        cb.fit(tr_X, tr_y)
        score = mean_absolute_error(val_y, cb.predict(val_X) * v1)
        scores.append(score)
    print("CB MAE: %0.4f (+/- %0.4f)" %(np.mean(scores), np.std(scores) * 2))
    md += 1

CB MAE: 37.0223 (+/- 11.8674)
CB MAE: 37.9318 (+/- 11.9344)
CB MAE: 38.2801 (+/- 12.0576)
CB MAE: 38.4831 (+/- 12.8829)
CB MAE: 38.2536 (+/- 13.5398)


In [249]:
data = test.copy()
feature = data.drop_duplicates(['단지코드', '총세대수'])[['단지코드', '지역', '총세대수', '공가수', '단지내주차면수']]
feature = pd.merge(feature, data.groupby('단지코드')['전용면적별세대수'].sum().reset_index(name = '총세대수2'), how = 'left', on = '단지코드')

In [250]:
for i in reg_list1:
    feature[i] = (feature['지역'] == i) * 1
feature['도시'] = 0

In [251]:
for i in reg_list2:
    feature['도시'] += feature[i]

In [252]:
#나이 비율
feature = pd.merge(feature, age, how = 'left', on = '지역')
for i in age_list2:
    feature[i] = 0
    for j in age_list3:
        feature[i] += feature[i + '(' + j + ')']

In [253]:
for i in age_list3:
    feature[i] = 0
    for j in age_list2:
        feature[i] += feature[j + '(' + i + ')']

In [254]:
for i in sup_list1:
    tmp = data[data['공급유형'] == i].groupby(['단지코드', '공급유형'])['전용면적별세대수'].sum().reset_index(name = i + '세대수')
    tmp[i] = 1
    tmp = tmp.drop('공급유형', axis = 1)
    feature = pd.merge(feature, tmp, how = 'left', on = '단지코드').fillna(0)

In [255]:
feature['공가비율'] = feature['공가수'] / feature['총세대수2']

In [256]:
tmp = data.copy()
tmp['총전용면적'] = tmp['전용면적'] * tmp['전용면적별세대수']
tmp = tmp.groupby(['단지코드'])['총전용면적'].sum().reset_index(name = '총전용면적')
feature = pd.merge(feature, tmp, how = 'left', on = '단지코드').fillna(0)
feature['평균전용면적'] = feature['총전용면적'] / feature['총세대수2']

In [257]:
tmp = data.copy()
tmp = tmp[tmp['임대건물구분'] == '상가']
tmp['상가총전용면적'] = tmp['전용면적'] * tmp['전용면적별세대수']
tmp = tmp.groupby(['단지코드'])['상가총전용면적'].sum().reset_index(name = '상가총전용면적')
feature = pd.merge(feature, tmp, how = 'left', on = '단지코드').fillna(0)
feature['상가평균전용면적'] = feature['상가총전용면적'] / feature['총세대수2']

In [258]:
tmp = data.copy()
tmp = tmp[tmp['임대건물구분'] == '아파트']
tmp['아파트총전용면적'] = tmp['전용면적'] * tmp['전용면적별세대수']
tmp = tmp.groupby(['단지코드'])['아파트총전용면적'].sum().reset_index(name = '아파트총전용면적')
feature = pd.merge(feature, tmp, how = 'left', on = '단지코드').fillna(0)
feature['아파트평균전용면적'] = feature['아파트총전용면적'] / feature['총세대수2']

In [259]:

for i in [1, 5, 10]:
    tmp = data.copy()
    tmp['전용면적'] = np.round(tmp['전용면적'] / i)
    tmp = tmp.groupby(['단지코드', '전용면적'])['전용면적별세대수'].sum().reset_index(name = '세대수')
    for j in range(10 // i, 600 // i + 1, 1):
        tmp2 = tmp[tmp['전용면적'] == j]
        tmp2['면적' + str(i) + '_' + str(j)] = tmp2['세대수']
        tmp2 = tmp2[['단지코드', '면적' + str(i) + '_' + str(j)]]
        feature = pd.merge(feature, tmp2, how = 'left', on = '단지코드').fillna(0)
feature = feature.drop('지역', axis = 1)
feature = feature.set_index('단지코드')

In [260]:
tmp = np.array(feature['단지내주차면수'])

feature = feature[feature_names_]

In [261]:
test_feature = np.array(feature)

In [262]:
random_state_ = 3
n_estimators_ = 1000
learning_rate_ = 0.002

train_X, train_y = train_feature, train_target

rgr = LGBMRegressor(n_estimators = 750,
                    random_state = 42,
                    learning_rate=learning_rate_)

rgr.fit(train_X, train_y)
lgbm_prediction = rgr.predict(test_feature)

lgbm_prediction = np.round(tmp * lgbm_prediction + tmp)

In [264]:
rf = RandomForestRegressor(n_estimators=1000, criterion = 'mae', random_state = 729)

rf.fit(train_X, train_y)
rf_prediction = rf.predict(test_feature)
rf_prediction = np.round(tmp * rf_prediction + tmp)

In [265]:
cb = CatBoostRegressor(iterations = 1000, max_depth = md, random_state = 42, learning_rate = .001, verbose = 0)

cb.fit(train_X, train_y)
cb_prediction = cb.predict(test_feature)
cb_prediction = np.round(tmp * cb_prediction + tmp)

In [266]:
submission = pd.read_csv("sample_submission.csv")

In [270]:
submission['num'] = (lgbm_prediction * .6  + cb_prediction * .4)

In [271]:
submission['num']

0       702.4
1      1294.8
2       477.0
3       529.6
4      1181.8
        ...  
145     264.4
146     483.2
147      39.6
148     378.2
149     258.0
Name: num, Length: 150, dtype: float64

In [272]:
submission.to_csv('0729.csv', index = False)